In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


## Define

In [ ]:
class HMM(object):
  def __init__(self):
    self.Ptrans = np.array([
                      [0.3, 0.3, 0.4], #Pstart
                      [0.1, 0.5, 0.4],
                      [0.3, 0.3, 0.4],
                      [0.4, 0.4, 0.2]
    ])
    self.Pvalues = np.array([
                      [0.20, 0.45, 0.20],
                      [0.25, 0.20, 0.10],
                      [0.55, 0.35, 0.70]
    ])
    self.values = {
        0: "Đỏ",
        1: "Vàng",
        2: "Xanh"
    }
    self.state_names = {
         0: "Start",
         1: "Bình 1",
         2: "Bình 2",
         3: "Bình 3"
    }
    self.current = 0
  
  def __next_state(self):
    prob = self.Ptrans[self.current,:]
    num_states = self.Ptrans.shape[0]-1
    state = np.random.choice(np.arange(1,num_states+1), size=1, p=prob)[0]
    self.current = state
    return state
  
  def __sample_on_state(self, state):
    prob = self.Pvalues[:, state-1]
    value = np.random.choice(list(self.values.values()), size=1, p=prob)[0]
    return value

  def sample(self, size):
    results =[]
    for c in np.arange(size):
      state = self.__next_state()
      value = self.__sample_on_state(state)
      results.append(value)
    return results
  
  def compute_prob(self, observations:list):

    alpha = np.empty([len(self.Ptrans)-1,len(observations)])

    O = self.Pvalues[list(self.values.keys())[list(self.values.values()).index(observations[0])]]
    alpha[:,0] = self.Ptrans[0].T*O
    
    for i in range(1, len(observations)):
      O = self.Pvalues[list(self.values.keys())[list(self.values.values()).index(observations[i])]]
      alpha[:,i] = (alpha[:,i-1]@self.Ptrans[1:].T)*O
      
    print(alpha)
    return np.sum(alpha[:,-1])


  def __init_vit(self,observations):
    delta = np.empty([len(self.Ptrans)-1,len(observations)])
    phi = np.empty([len(self.Ptrans)-1,len(observations)])
    O = self.Pvalues[list(self.values.keys())[list(self.values.values()).index(observations[0])]]

    delta[:,0] = self.Ptrans[0].T*O
    phi[:,0] = -1

    return delta,phi

  def __loop_vit(self,observations,delta,phi):

    for i in range(1, len(observations)):
      O = self.Pvalues[list(self.values.keys())[list(self.values.values()).index(observations[i])]]
      delta[:,i] = np.max(delta[:,i-1]*self.Ptrans[1:].T*O.T,1)
      phi[:,i] =  np.argmax(delta[:,i-1]*self.Ptrans[1:].T,1) 
    return delta,phi

  def __prob_path_vit(self,loc,delta,phi):
    prob = np.max(delta[:,loc-1])
    q = np.argmax(delta[:,loc-1])
    
    path = np.empty(loc, 'B')
    path[-1] = np.argmax(delta[:, loc -1])
    for i in reversed(range(1, loc)):
        path[i - 1] = phi[path[i], i]

    return prob,path

  def viterbi(self, observations:list,table= True):

    
    delta,phi = self.__init_vit(observations)
    delta,phi = self.__loop_vit(observations,delta,phi)
    
    loc = len(observations)
    prob, path = self.__prob_path_vit(loc,delta,phi)
    if table:
      print('--'*50)
      print("Delta:")
      print(pd.DataFrame(delta))
      print('--'*50)
      print("Phi:")
      print(pd.DataFrame(phi))
      print('--'*50)

    return prob, path

##Using

In [ ]:
model = HMM()

In [ ]:
L = model.sample(10)
L

['Đỏ', 'Đỏ', 'Xanh', 'Xanh', 'Xanh', 'Xanh', 'Xanh', 'Xanh', 'Xanh', 'Xanh']

In [ ]:
P = model.compute_prob(['Vàng', 'Đỏ', 'Đỏ', 'Đỏ'])
print(P)

[[0.075      0.0107     0.0037485  0.00105674]
 [0.06       0.025425   0.00710887 0.00207523]
 [0.04       0.0124     0.003386   0.00100403]]
0.004135993125


In [ ]:
prob, path = model.viterbi(L)
print(f'Prob: {prob}')
print('--'*50)
print(f'Path: {path}')


----------------------------------------------------------------------------------------------------
Delta:
       0         1         2         3         4         5         6  \
0  0.060  0.018225  0.006804  0.001123  0.000419  0.000073  0.000026   
1  0.135  0.018225  0.006804  0.001871  0.000419  0.000115  0.000026   
2  0.080  0.024300  0.004010  0.001497  0.000262  0.000092  0.000016   

          7             8             9  
0  0.000005  1.590400e-06  2.783200e-07  
1  0.000007  1.590400e-06  4.373601e-07  
2  0.000006  9.940002e-07  3.498881e-07  
----------------------------------------------------------------------------------------------------
Phi:
     0    1    2    3    4    5    6    7    8    9
0 -1.0  1.0  2.0  1.0  2.0  1.0  2.0  1.0  2.0  1.0
1 -1.0  1.0  2.0  0.0  2.0  0.0  2.0  0.0  2.0  0.0
2 -1.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0  1.0  0.0
----------------------------------------------------------------------------------------------------
Prob: 4.373600749056